In [4]:
import requests
from requests.auth import HTTPBasicAuth

client_id = "CF15468A353C46588071B43CC1636C67"
client_secret = "UK2LT80ZGozBy7ZHEf-oRP5hD6vgdPo6kQa-5QvOoIpjWEPf"
redirect_uri = "https://xero.com"
authorization_code = "PbPXrqXW62nUn8eZnUd3shpXMTFG4kR_STxbkCYwq7A"

response = requests.post(
    "https://identity.xero.com/connect/token",
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={
        "grant_type": "authorization_code",
        "code": authorization_code,
        "redirect_uri": redirect_uri,
    },
    auth=HTTPBasicAuth(client_id, client_secret)
)

# Print the response to check for errors
print(response.json())

tokens = response.json()
access_token = tokens.get("access_token")
refresh_token = tokens.get("refresh_token")

if not access_token:
    print("Failed to retrieve access token. Response:", tokens)
else:
    print("Access Token:", access_token)
    print("Refresh Token:", refresh_token)


{'id_token': 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjFDQUY4RTY2NzcyRDZEQzAyOEQ2NzI2RkQwMjYxNTgxNTcwRUZDMTkiLCJ0eXAiOiJKV1QiLCJ4NXQiOiJISy1PWm5jdGJjQW8xbkp2MENZVmdWY09fQmsifQ.eyJuYmYiOjE3MzE2Njk5NTEsImV4cCI6MTczMTY3MDI1MSwiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS54ZXJvLmNvbSIsImF1ZCI6IkNGMTU0NjhBMzUzQzQ2NTg4MDcxQjQzQ0MxNjM2QzY3IiwiaWF0IjoxNzMxNjY5OTUxLCJhdF9oYXNoIjoiNncwZFN2ZWNEN1BMdWV1NWFibVF3QSIsInNfaGFzaCI6InBtV2tXU0JDTDUxQmZraG43OXhQdUEiLCJzaWQiOiI1ODc2NDJlYjE1Mzg0YjI5YTgxZTQxYzQ4OWNmOWNlNyIsInN1YiI6IjVlZmM2NWNlMDFmZjVlOGE5MzJmMWQ4YTY4ODJkM2JjIiwiYXV0aF90aW1lIjoxNzMxNjY3NTg3LCJ4ZXJvX3VzZXJpZCI6IjE2MDFkM2VjLTI4ZDUtNDFhMS04Mjk0LTc2ODNhNGJlZWVhZiIsImdsb2JhbF9zZXNzaW9uX2lkIjoiNTg3NjQyZWIxNTM4NGIyOWE4MWU0MWM0ODljZjljZTciLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJucjc1MTYzMkBnbWFpbC5jb20iLCJlbWFpbCI6Im5yNzUxNjMyQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJOaWtoaWwiLCJmYW1pbHlfbmFtZSI6IlJhbyIsIm5hbWUiOiJOaWtoaWwgUmFvIiwiYW1yIjpbInB3ZCJdfQ.Q4uXlTNhkVUtg7r583v6Sfc7HZsKqjIozE-38IJ7JdSNCNmIrriNG9BOCWkl9aN-IFyC5gkdBwuceNps-_RRjAfAFA8j

In [5]:
response = requests.get(
    "https://api.xero.com/connections",
    headers={
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
)
connections = response.json()
tenant_id = connections[0]["tenantId"]

In [6]:
response = requests.get(
    "https://api.xero.com/connections",
    headers={
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
)

connections = response.json()
print("Response from connections endpoint:", connections)

if connections:
    tenant_id = connections[0]["tenantId"]
    print("Tenant ID:", tenant_id)
else:
    print("No tenant connections found.")

Response from connections endpoint: [{'id': 'c414a6ca-70f0-4536-9977-f7ca41cd855a', 'authEventId': '2d126555-6f4b-4ded-aa8d-734591ff5040', 'tenantId': '2820b5d5-fc3b-4f8c-9ecc-213482005224', 'tenantType': 'ORGANISATION', 'tenantName': 'Xero automation', 'createdDateUtc': '2024-11-15T05:55:05.2832940', 'updatedDateUtc': '2024-11-15T11:16:00.0835600'}]
Tenant ID: 2820b5d5-fc3b-4f8c-9ecc-213482005224


In [ ]:
import requests
import json
import time

# Xero API credentials
xero_client_id = "CF15468A353C46588071B43CC1636C67"
xero_client_secret = "UK2LT80ZGozBy7ZHEf-oRP5hD6vgdPo6kQa-5QvOoIpjWEPf"
xero_tenant_id = "2820b5d5-fc3b-4f8c-9ecc-213482005224"
xero_refresh_token = "u9fFPDzhxr_vL4rCdO5BRcosP6I97B7zYwxVEnzbs9M"

# Slack Webhook URL
slack_webhook_url = "https://nikhilrao1234.app.n8n.cloud/webhook-test/03ff7704-7ff0-4f32-b220-531b783fc5e5"

def get_xero_access_token():
    url = "https://identity.xero.com/connect/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "refresh_token",
        "client_id": xero_client_id,
        "client_secret": xero_client_secret,
        "refresh_token": xero_refresh_token,
    }
    response = requests.post(url, headers=headers, data=data)
    response_data = response.json()
    return response_data["access_token"]

def check_for_new_invoices():
    access_token = get_xero_access_token()
    url = f"https://api.xero.com/api.xro/2.0/Invoices"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Xero-tenant-id": xero_tenant_id,
        "Accept": "application/json",
    }
    response = requests.get(url, headers=headers)
    invoices = response.json().get("Invoices", [])
    return invoices

def check_for_new_payments():
    access_token = get_xero_access_token()
    url = f"https://api.xero.com/api.xro/2.0/Payments"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Xero-tenant-id": xero_tenant_id,
        "Accept": "application/json",
    }
    response = requests.get(url, headers=headers)
    payments = response.json().get("Payments", [])
    return payments

def send_slack_notification(message):
    payload = {"text": message}
    requests.post(slack_webhook_url, data=json.dumps(payload), headers={"Content-Type": "application/json"})

def monitor_xero():
    while True:
        invoices = check_for_new_invoices()
        payments = check_for_new_payments()
        
        for invoice in invoices:
            invoice_message = f"New invoice created: {invoice['InvoiceNumber']} for {invoice['Contact']['Name']}"
            send_slack_notification(invoice_message)
        
        for payment in payments:
            payment_message = f"New payment received: {payment['PaymentID']} for {payment['Invoice']['InvoiceNumber']}"
            send_slack_notification(payment_message)
        
        time.sleep(300)  # Check every 5 minutes

if __name__ == "__main__":
    monitor_xero()

In [ ]:
import requests
import json
import time

# Xero API credentials
xero_client_id = "CF15468A353C46588071B43CC1636C67"
xero_client_secret = "UK2LT80ZGozBy7ZHEf-oRP5hD6vgdPo6kQa-5QvOoIpjWEPf"
xero_tenant_id = "2820b5d5-fc3b-4f8c-9ecc-213482005224"
xero_refresh_token = "u9fFPDzhxr_vL4rCdO5BRcosP6I97B7zYwxVEnzbs9M"

# Slack Webhook URL
slack_webhook_url = "https://nikhilrao1234.app.n8n.cloud/webhook-test/03ff7704-7ff0-4f32-b220-531b783fc5e5"

def get_xero_access_token():
    url = "https://identity.xero.com/connect/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "refresh_token",
        "client_id": xero_client_id,
        "client_secret": xero_client_secret,
        "refresh_token": xero_refresh_token,
    }
    response = requests.post(url, headers=headers, data=data)
    response_data = response.json()
    print("Access Token Response:", response_data)
    return response_data["access_token"]

def check_for_new_invoices():
    access_token = get_xero_access_token()
    url = f"https://api.xero.com/api.xro/2.0/Invoices"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Xero-tenant-id": xero_tenant_id,
        "Accept": "application/json",
    }
    response = requests.get(url, headers=headers)
    invoices = response.json().get("Invoices", [])
    print("Invoices Retrieved:", invoices)
    return invoices

def check_for_new_payments():
    access_token = get_xero_access_token()
    url = f"https://api.xero.com/api.xro/2.0/Payments"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Xero-tenant-id": xero_tenant_id,
        "Accept": "application/json",
    }
    response = requests.get(url, headers=headers)
    payments = response.json().get("Payments", [])
    print("Payments Retrieved:", payments)
    return payments

def send_slack_notification(message):
    payload = {"text": message}
    response = requests.post(slack_webhook_url, data=json.dumps(payload), headers={"Content-Type": "application/json"})
    print("Slack Notification Response:", response.status_code, response.text)

def monitor_xero():
    while True:
        invoices = check_for_new_invoices()
        payments = check_for_new_payments()
        
        for invoice in invoices:
            invoice_message = f"New invoice created: {invoice['INV-0001']} for {invoice['9739501114']['nisha rao']}"
            print("Sending Invoice Notification:", invoice_message)
            send_slack_notification(invoice_message)
        
        for payment in payments:
            payment_message = f"New payment received: {payment['PaymentID']} for {payment['Invoice']['InvoiceNumber']}"
            print("Sending Payment Notification:", payment_message)
            send_slack_notification(payment_message)'''
        
        time.sleep(300)  # Check every 5 minutes

if __name__ == "__main__":
    monitor_xero()

In [ ]:
import requests
import json
import time
import os
from requests.auth import HTTPBasicAuth

# Xero and Slack credentials from environment variables
XERO_CLIENT_ID = os.getenv("CF15468A353C46588071B43CC1636C67")
XERO_CLIENT_SECRET = os.getenv("UK2LT80ZGozBy7ZHEf-oRP5hD6vgdPo6kQa-5QvOoIpjWEPf")
XERO_TENANT_ID = os.getenv('"2820b5d5-fc3b-4f8c-9ecc-213482005224"')
XERO_REFRESH_TOKEN = os.getenv("u9fFPDzhxr_vL4rCdO5BRcosP6I97B7zYwxVEnzbs9M")
SLACK_WEBHOOK_URL = os.getenv('https://nikhilrao1234.app.n8n.cloud/webhook-test/03ff7704-7ff0-4f32-b220-531b783fc5e5')

# Temporary storage for pending invoices (in memory for simplicity)
pending_invoices = {}

def get_xero_access_token():
    # Function to refresh and get a new access token from Xero
    token_url = "https://identity.xero.com/connect/token"
    token_data = {
        "grant_type": "refresh_token",
        "refresh_token": XERO_REFRESH_TOKEN,
    }
    response = requests.post(token_url, data=token_data, auth=HTTPBasicAuth(XERO_CLIENT_ID, XERO_CLIENT_SECRET))
    if response.status_code == 200:
        tokens = response.json()
        return tokens['access_token']
    else:
        raise Exception(f"Failed to get access token: {response.text}")

def fetch_invoices(access_token):
    # Function to fetch invoices from Xero
    invoice_url = f"https://api.xero.com/api.xro/2.0/Invoices"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Xero-tenant-id": XERO_TENANT_ID,
    }
    response = requests.get(invoice_url, headers=headers)
    if response.status_code == 200:
        return response.json()["Invoices"]
    else:
        raise Exception(f"Failed to fetch invoices: {response.text}")

def fetch_payments(access_token):
    # Function to fetch payments from Xero
    payment_url = f"https://api.xero.com/api.xro/2.0/Payments"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Xero-tenant-id": XERO_TENANT_ID,
    }
    response = requests.get(payment_url, headers=headers)
    if response.status_code == 200:
        return response.json()["Payments"]
    else:
        raise Exception(f"Failed to fetch payments: {response.text}")

def send_slack_notification(message):
    payload = {'text': message}
    response = requests.post(SLACK_WEBHOOK_URL, data=json.dumps(payload), headers={'Content-Type': 'application/json'})
    if response.status_code != 200:
        raise Exception(f"Request to Slack returned an error {response.status_code}, the response is:\n{response.text}")

def main():
    while True:
        try:
            # Refresh access token
            access_token = get_xero_access_token()
            
            # Fetch and send notifications for new invoices
            invoices = fetch_invoices(access_token)
            for invoice in invoices:
                if invoice['Status'] == 'AUTHORISED':  # Check if the invoice is authorised
                    pending_invoices[invoice['Invoice']] = invoice
                    send_slack_notification(f"New Invoice Created: {invoice['InvoiceNumber']}")
            
            # Fetch payments and update pending invoices status
            payments = fetch_payments(access_token)
            for payment in payments:
                invoice_number = payment['Invoice']['InvoiceNumber']
                if invoice_number in pending_invoices:
                    send_slack_notification(f"Payment Received: {payment['PaymentID']} for Invoice: {invoice_number}")
                    del pending_invoices[invoice_number]
            
            # Sleep for a specified time before checking again
            time.sleep(300)  # Check every 5 minutes
        except Exception as e:
            print(f"An error occurred: {e}")
            time.sleep(60)  # Wait for a minute before retrying

if __name__ == "__main__":
    main()